In [1]:
import pandas as pd
def to_dataframe(texts):
    ids = []
    labels = []
    parents = []
    info = []
    levels = []

    for idx, text in enumerate(texts):
        parts = text.split('-')
        for i, part in enumerate(parts):
            ids.append("-".join(parts[:i+1]))
            labels.append(part)
            parents.append("-".join(parts[:i]) if i > 0 else "")
            info.append(part)
            levels.append(i+1)
    return pd.DataFrame({
        "ids": ids,
        "labels": labels,
        "parents": parents,
        "info": info,
        "level": levels
    })


In [46]:
with open('example2.txt', 'r') as f:
    example = f.read()

input_list = example.split('\n')
print(input_list)
input_list = to_dataframe(input_list)
print(input_list)

['Chatbots - Chatbots are AI-based software designed to interact with humans in their natural languages. - These chatbots are commonly used in industries for customer service, request routing, or for information gathering tasks. They use natural language processing (NLP) and machine learning algorithms to understand and respond to human inputs, making them an effective tool for 24/7 customer support.', "Recommendation Systems - Recommendation systems are a popular AI application, transforming the way online businesses engage with customers. - They analyze a user's behavior, preferences, and interaction history to provide personalized product or content suggestions. This technology is widely used in sectors such as eCommerce, entertainment (Netflix's movie recommendations, for example), and digital advertising.", 'Autonomous Vehicles - Autonomous vehicles leverage AI technology to navigate and understand the environment without human input. - These vehicles use a combination of sensors,

In [49]:
import textwrap
def auto_wrap_text(text, max_width):
    wrapped_text = textwrap.wrap(text, max_width)
    wrapped_text = "<br>".join(wrapped_text)
    return wrapped_text

def apply_wrap(row):
    if row['level'] == 1:
        return auto_wrap_text(row['labels'], max_width=30)
    elif row['level'] == 2:
        return auto_wrap_text(row['labels'], max_width=50)
    else:
        return auto_wrap_text(row['labels'], max_width=80)


# 尝试不同的内容的换行
# 不同level的换行

In [50]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import json
import textwrap
import numpy as np
import plotly.colors as colors
import matplotlib.pyplot as plt
import random
import colorsys

# 定义颜色刻度，根据 level 的最小值和最大值生成渐变色
color_scale = colors.sequential.Blues

# 生成模拟数据


df = pd.DataFrame(input_list)

# df["labels"] = df["labels"].apply(lambda x: auto_wrap_text(x, max_width=20))  # 20 是你想要的最大行长度
df["labels"] = df.apply(apply_wrap, axis=1)

# 根据 level 的值定义三个蓝色
# color_scale = colors.sequential.Blues
# 定义颜色刻度，根据 level 的最小值和最大值生成渐变色
# color_scale = [[0, 'rgb(66, 135, 245)'], [0.5, 'rgb(135, 183, 245)'], [1, 'rgb(183, 215, 245)']] # 蓝色
# color_scale = [[0, 'rgb(66, 135, 245)'], [0.25, 'rgb(135, 183, 245)'], [0.5, 'rgb(183, 215, 245)'], [0.75, 'rgb(215, 231, 245)'], [1, 'rgb(245, 245, 245)']]
# # 计算每个标签的字数
# label_lengths = df['labels'].apply(len)



# 选择一个基本颜色（这里是蓝绿色，对应HSL色调的180度）
base_hue = random.random()  # 在Python的colorsys模块中，色调的范围是0-1，所以我们需要把角度除以360
# random.seed(0)


# 生成色系相似的8个颜色
base_colors = []
for i in range(8):
    # 在色调上加上一个小的偏移（在这里是i*10度），并确保结果在0-1范围内
    hue = (base_hue + i*10/360) % 1
    # 使用饱和度1和亮度0.5生成颜色（这将生成一个鲜艳的颜色）
    color = colorsys.hls_to_rgb(hue, 0.5, 1)
    # 将颜色从RGB空间转换到十六进制颜色代码
    base_colors.append("#"+''.join([f"{int(val*255):02x}" for val in color]))
    

# 转换十六进制颜色代码为RGB颜色值（范围是0-1）
base_colors_rgb = [[int(color[i:i+2], 16) / 255 for i in range(1, 6, 2)] for color in base_colors]

# 节点和层级的关系，假设我们有8个段落，每个段落有3个层级
nodes = ["P{}-L{}".format(p, l) for p in range(8) for l in range(3)]
levels = [l for p in range(8) for l in range(3)]

# 根据节点的层级和对应的基础颜色来生成颜色
node_colors = []
for i in range(8):
    for j in range(3):
        # 随着层级增加，将颜色的亮度增加
        h, l, s = colorsys.rgb_to_hls(*base_colors_rgb[i])
        color = colorsys.hls_to_rgb(h, min(1, l + 0.2*j), s)  # 每次增加20%的亮度，最大为1
        node_colors.append("rgba"+str(tuple([int(255*val) for val in color] + [1.0])) + ")")




fig = go.Figure(go.Treemap(
    ids=df['ids'],
    labels=df['labels'],
    parents=df['parents'],
    # values=df['level'],
    # values=[10]*len(df['level']),
    maxdepth=2,
    # text=df['labels'],
    insidetextfont=dict(size=30, color='#FFFFFF'),
    outsidetextfont=dict(size=15, color='#FF0000', family='Arial'),
    # hoverlabel=dict(
    #     font=dict(
    #         color='#000000',
    #         size=8,
    #         family='Arial'
    #     ),
    #     bgcolor='lightyellow'
    # ),
    # pathbar_textfont_size=15,
    # textfont=dict(size=20, exponent=-1) 
    
    # marker_colorscale = 'Blues',  # 使用 'Blues' 颜色方案
    marker_colors=node_colors,  # 使用标签长度作为权重
    # marker_colorscale=color_scale,  # 使用自定义的颜色刻度
    
    
    # tiling = dict(packing='slice-dice')  # 使用 'slice-dice' 布局算法
    tiling=dict(
    packing='squarify',  # Set packing algorithm to 'squarify'
    pad=0.2
    )
))

fig.update_layout(
    # title="人工智能在不同行业的应用",  # 添加标题
    title={
    'text': "<b>人工智能在不同行业的应用",
    'x': 0.5,  # 将标题的 x 位置设置为 0.5，即图形的中心
    'xanchor': 'center',  # 将标题的 x 锚点设置为 'center'，即标题的中心
    # 'y' : 0.96 , # 高度调整
    },
    title_font=dict(
    family="Courier New, monospace",  # 设置字体
    size=20,  # 设置字体大小
    color="RebeccaPurple"  # 设置字体颜色
    ),
    # uniformtext=dict(minsize=15, mode='hide'), 
    margin = dict(t=50, l=25, r=25, b=25),
    width=600,
    height=600,
    # plot_bgcolor="#0000FF",  # 设置画布的背景颜色
    # autosize=True,
)

# fig.update_layout(
#     # 
# )

fig.update_traces(marker=dict(cornerradius=20)) # 圆角
fig.show()


# app = JupyterDash(__name__)
# 召回函数
# app.layout = html.Div([
#     dcc.Graph(
#         id='my-graph',
#         figure=fig
#     ),
#     # html.Pre(id='click-data', style={'padding': '10px'}),
#     # html.Div(id='text-storage', style={'display': 'none'}),
#     # html.Div(id='node-clicked', style={'display': 'none'})
# ])

# @app.callback(
#     Output('my-graph', 'figure'),
#     Output('text-storage', 'children'),
#     Output('node-clicked', 'children'),
#     Input('my-graph', 'clickData'),
#     State('my-graph', 'figure'),
#     State('text-storage', 'children'),
#     State('node-clicked', 'children')
# )

# def update_figure(clickData, fig, stored_text, clicked_node):
#     if clickData:
#         point_number = clickData['points'][0]['pointNumber']
#         if stored_text is None:
#             stored_text = json.dumps(fig['data'][0]['text'])
#         if clicked_node is None or clicked_node != point_number:
#             fig['data'][0]['text'] = json.loads(stored_text)
#             fig['data'][0]['insidetextfont']['size'] = 30  # 将字体大小设置为30
#             clicked_node = point_number
#         else:
#             fig['data'][0]['text'] = json.loads(stored_text)
#             fig['data'][0]['insidetextfont']['size'] = 20  # 将字体大小重设为20
#             clicked_node = None
#     else:
#         if stored_text is not None:
#             fig['data'][0]['text'] = json.loads(stored_text)
#         fig['data'][0]['insidetextfont']['size'] = 20  # 将字体大小重设为20
#         clicked_node = None

#     return fig, stored_text, clicked_node


# app.run_server(mode='inline', port=8054)


In [7]:
len(node_colors)

24

In [30]:
base_hue = 180/360  # 在Python的colorsys模块中，色调的范围是0-1，所以我们需要把角度除以360
base_hue, random.random()
# get a random 0-1 number





(0.5, 0.7579544029403025)

In [164]:
df = pd.DataFrame(input_list)
df

,ids,labels,parents,info,level
0,1.虚拟现实VR,1.虚拟现实VR,,1.虚拟现实VR,1
1,1.虚拟现实VR-虚拟现实是一种通过计算机生成的模拟环境，使用户能够与虚拟世界进行互动和沉浸式体验,虚拟现实是一种通过计算机生成的模拟环境，使用户能够与虚拟世界进行互动和沉浸式体验,1.虚拟现实VR,虚拟现实是一种通过计算机生成的模拟环境，使用户能够与虚拟世界进行互动和沉浸式体验,2
2,1.虚拟现实VR-虚拟现实是一种通过计算机生成的模拟环境，使用户能够与虚拟世界进行互动和沉浸...,虚拟现实（VR）是一种通过计算机技术创造的模拟环境，让用户能够身临其境地感受和探索虚拟世界。...,1.虚拟现实VR-虚拟现实是一种通过计算机生成的模拟环境，使用户能够与虚拟世界进行互动和沉浸式体验,虚拟现实（VR）是一种通过计算机技术创造的模拟环境，让用户能够身临其境地感受和探索虚拟世界。...,3
3,2.区块链技术,2.区块链技术,,2.区块链技术,1
4,2.区块链技术-区块链是一种分布式的账本技术，通过去中心化的方式实现信息的安全存储和交易的可追溯性,区块链是一种分布式的账本技术，通过去中心化的方式实现信息的安全存储和交易的可追溯性,2.区块链技术,区块链是一种分布式的账本技术，通过去中心化的方式实现信息的安全存储和交易的可追溯性,2
5,2.区块链技术-区块链是一种分布式的账本技术，通过去中心化的方式实现信息的安全存储和交易的可...,区块链技术是一种基于密码学原理的分布式账本技术，它通过去中心化的方式将交易信息记录在一个不断...,2.区块链技术-区块链是一种分布式的账本技术，通过去中心化的方式实现信息的安全存储和交易的可追溯性,区块链技术是一种基于密码学原理的分布式账本技术，它通过去中心化的方式将交易信息记录在一个不断...,3
6,3.物联网（IoT）,3.物联网（IoT）,,3.物联网（IoT）,1
7,3.物联网（IoT）-物联网是将各种设备、传感器和物体连接到互联网，实现智能化和自动化的网络系统。,物联网是将各种设备、传感器和物体连接到互联网，实现智能化和自动化的网络系统。,3.物联网（IoT）,物联网是将各种设备、传感器和物体连接到互联网，实现智能化和自动化的网络系统。,2
8,3.物联网（IoT）-物联网是将各种设备、传感器和物体连接到互联网，实现智能化和自动化的网络...,物联网（Internet of Things，IoT）是一种将传感器、设备和物体连接到互联网...,3.物联网（IoT）-物联网是将各种设备、传感器和物体连接到互联网，实现智能化和自动化的网络系统。,物联网（Internet of Things，IoT）是一种将传感器、设备和物体连接到互联网...,3
9,4.自动驾驶,4.自动驾驶,,4.自动驾驶,1
